In [22]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv
import time


#### I made multiple csvs...
- gpt_answer_v1.csv - model: text-curie-001 - using prompts from  reddit_questions.csv - 4615 rows - Quick proof of concept. Really Dirty Data. 
     
        -TrueAskReddit
        -AskHistorians
        -AskScienceFiction
        -NoStupidQuestions
        -relationship_advice
        -AskProgramming
        -AskAcademia
        -AskPhilosophy
        -AskMen
        -AskWomen
        -AskStatistics
- gpt_answer_v2.csv - model: text-curie-001 - using prompts from  reddit_questions3.csv and reddit_questions4.csv - 5303 rows - Used for project


        - AskAcademia
        - AskPhilosophy
        - Ask_Politics
        - AskGames
        - AskStatistics
        - answers
        - AskAnthropology
        - relationship_advice
        - AskSocialScience
        - AskScience
        - AskWomen
        - AskMen
        - AskScience
        - AskReddit        
- gpt_answer_v3.csv - model: text-curie-001 - using prompts from  reddit_questions5.csv - 2187 rows - Used for project


        - NoStupidQuestions
        - AskHistorians
        - TrueAskReddit
        - AskProgramming


In [40]:
df3 = pd.read_csv('data/reddit_questions3.csv')
df4 = pd.read_csv('data/reddit_questions4.csv')
df5 = pd.read_csv('data/reddit_questions5.csv')

In [41]:
df3.drop(columns=['Unnamed: 0'], inplace = True)
df4.drop(columns=['Unnamed: 0'], inplace = True)
df5.drop(columns=['Unnamed: 0'], inplace = True)

In [42]:
df3.shape

(1136, 3)

In [43]:
df4.shape

(4168, 3)

In [44]:
df5.shape

(2188, 3)

In [10]:
#d = pd.concat([df3, df4], ignore_index=True)

I'd like to thank Alan for the great suggestion to use the dotenv library so that I can continue to work in jupyter lab.

In [45]:
load_dotenv()  

True

In [46]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [47]:
df5.head()

,Subreddit,Questions,Human_Answer
0,NoStupidQuestions,"Why is it ""price gouging"" when people resell s...",Who said either of those was ok?
1,NoStupidQuestions,Does anyone else stay up at night because they...,Nearly every night. I don’t want life to just...
2,NoStupidQuestions,Does anybody else hate how the Internet now fe...,Stumbleupon was awesome a decade ago.
3,NoStupidQuestions,Does anyone else stay up at night because that...,Yep
4,NoStupidQuestions,Is it weird that my 35 year old friend is wait...,"Meanwhile, in r/teenagers, there’s a 17 year o..."


Here is where I utilize openai's soon to be deprecated, Completion class to get my responses from chatgpt. I use the 'text-curie-001' model for cheap and fast execution. I'd like to thank Vijay for suggesting the batch size of 20. I need to do more research as to why he thinks that's the max batch size that completion could take. I'd also like to thank Eric again, for his breakfast hour class. 

Basically, I chunk up all the questions into batches, iterate through those batches, get the responses from each batch and put the output in a list (response_list)

In [48]:
# I could just use the Series object, but I feel more comfortable with lists
questions = [question for question in df5['Questions']]
# I was advised to use 20 as a batch size(I was advised that it was the max that Completion could take, but I couldnt find any info on it)
batch_size = 20
# Chunk up questions into a list of lists each inner list is of size batch size(20). The last list is the remainder.
question_batches = [questions[i:i + batch_size] for i in range(0, len(questions), batch_size)]
# Storage for the responses 
response_list = []
# count and num_batches are just for me to measure time
count = 0
num_batches = len(question_batches)

# go through each batch, and get the responses
for batch in question_batches:
    
    # This is thanks to Eric from the breakfast hour
    maries_response = openai.Completion.create(
        model = 'text-curie-001', # good, cheap model
        prompt = batch,
        temperature = 0.6, # may want to play with this in the future
        max_tokens = 250 # After some experimentation, I was satisfied with this
    )
   
    # Just want a list of the responses -- nothing else. 
    # Hope is what is keeping the response order the same as the question order. 
    # But it isn't necessary for my implementation of the project. I'll explore in the future
    batch_response = [res['text'].strip() for res in maries_response['choices']]
    
    # Append to the big list of responses
    response_list += batch_response
    

    #sleep time
    time.sleep(1)
    #This is just for me to see progress
    count+=1
    print(f'finished batch number {count}/{num_batches}')


finished batch number 1/110
finished batch number 2/110
finished batch number 3/110
finished batch number 4/110
finished batch number 5/110
finished batch number 6/110
finished batch number 7/110
finished batch number 8/110
finished batch number 9/110
finished batch number 10/110
finished batch number 11/110
finished batch number 12/110
finished batch number 13/110
finished batch number 14/110
finished batch number 15/110
finished batch number 16/110
finished batch number 17/110
finished batch number 18/110
finished batch number 19/110
finished batch number 20/110
finished batch number 21/110
finished batch number 22/110
finished batch number 23/110
finished batch number 24/110
finished batch number 25/110
finished batch number 26/110
finished batch number 27/110
finished batch number 28/110
finished batch number 29/110
finished batch number 30/110
finished batch number 31/110
finished batch number 32/110
finished batch number 33/110
finished batch number 34/110
finished batch number 3

In [50]:
len(response_list)

2188

Attaching the responses to the dataframe. Like I mentioned in the comments of the code, the question-answer pairs for the ai generated responses arent guarenteed to match up. I'm pretty sure they are, but I can't 100% guarentee that(I'd have to do more research on how COmpletions handles batch prompts). 

**However**, this question-answer pairing is NOT VITAL to my GA project goal. I just want to differentiate between chatgpt and human. the questions were just a convenient way for me to get an equal amount of training/test data for both classes.    

In [51]:
df5['Marie_Answer'] = response_list

In [52]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2188 entries, 0 to 2187
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Subreddit     2188 non-null   object
 1   Questions     2188 non-null   object
 2   Human_Answer  2188 non-null   object
 3   Marie_Answer  2188 non-null   object
dtypes: object(4)
memory usage: 68.5+ KB


In [53]:
df5.head()

,Subreddit,Questions,Human_Answer,Marie_Answer
0,NoStupidQuestions,"Why is it ""price gouging"" when people resell s...",Who said either of those was ok?,One possible reason why it might be considered...
1,NoStupidQuestions,Does anyone else stay up at night because they...,Nearly every night. I don’t want life to just...,I stay up at night because I have a lot of fre...
2,NoStupidQuestions,Does anybody else hate how the Internet now fe...,Stumbleupon was awesome a decade ago.,
3,NoStupidQuestions,Does anyone else stay up at night because that...,Yep,"I do not know what you mean by ""stay up at nig..."
4,NoStupidQuestions,Is it weird that my 35 year old friend is wait...,"Meanwhile, in r/teenagers, there’s a 17 year o...",It is not weird.


Writing to csv

In [54]:
df5.to_csv('data/gpt_answers_v3.csv')